In [ ]:
import descarteslabs as dl
import ee
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import shapely.geometry

In [ ]:
def get_elevation(country_code, threshold=400, save=True, plot=True):
    ee.Initialize()

    dataset = ee.Image('CGIAR/SRTM90_V4')

    country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_co', country_code))
    country_bounds = gpd.GeoDataFrame.from_features(country.getInfo()['features'], crs='WGS84')

    data = dataset.select('elevation').clipToCollection(country)
    zones = data.gt(threshold);
    zones = zones.updateMask(zones.neq(0));

    vectors = zones.addBands(data).reduceToVectors(
        geometry = country,
        crs = data.projection(),
        scale = 30,
        geometryType = 'polygon',
        eightConnected = True,
        labelProperty = 'zone',
        reducer = ee.Reducer.mean())

    url = vectors.getDownloadURL()
    df = pd.read_csv(url)
    geoms = [shapely.geometry.Polygon(json.loads(g)['coordinates'][0]) for g in df['.geo']]
    polygons = gpd.GeoDataFrame(geometry=geoms, crs='EPSG:4326')

    area = polygons.to_crs('epsg:3857')['geometry'].area.sum() / 10 ** 6
    country_area = (country_bounds.to_crs('epsg:3857')['geometry'].area / 10 ** 6).values[0]
    print(f"{area / country_area:.3%}")

    if plot:
        fig, ax = plt.subplots(figsize=(8,5), dpi=150)
        country_bounds.plot(ax=ax)
        polygons.plot(ax=ax, color='r')
        plt.xticks([])
        plt.yticks([])
        plt.title(f"{country_code}, Elev Threshold {threshold}")
        plt.show()
    if save:
        polygons.to_file(f'../../data/boundaries/{country_code.lower()}_elev_{threshold}.geojson', driver='GeoJSON')
    return polygons

In [ ]:
get_elevation('BP', threshold=400, save=True, plot=True)